# **Import delle librerie**

In [26]:
import math
import numpy as np
import os, glob, cv2
import mediapipe as mp
import pandas
import LipLandmarks as lp
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from tqdm import tqdm
import matplotlib.pyplot as plt

# Esempio di Lip Feature Extraction

Per estrapolare dati associati alle caratteristiche biometriche delle labbra dobbiamo riconscere se all'interno di una data immaginare è presente o meno un volto (Face Detection)

Per questo faremo ricorso all'uso della libreria MediaPipe.

Ecco una funzione che usa la libreria MediaPipe per restuire il risultato della face detection di un immagine:

In [29]:
# Inizializza la libreria Mediapipe per la face mesh
def detect_face(image_name):
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh()

    # Carica l'immagine di esempio di Obama
    sample_image = cv2.imread("obama.jpg")

    # Converte l'immagine in RGB
    rgb_frame = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)

    # Processa l'immagine con Mediapipe per ottenere i punti della face mesh
    results = face_mesh.process(rgb_frame)
    face_landmarks = results.multi_face_landmarks
    
    return results, sample_image


Quando la face detection va a buon fine, cioè quando MediaPipe è in grado di riconoscere la presenza del volto in una data immagine, individua la posizione di 478 landmarks di rilevante importanza biometrica.

Ecco un esempio di face detection su un'immagine usando la funzione precedente.

Il risultato della funzione usato per mostare sull'immagine del volto i landmark individuati nella fase di face detection

In [ ]:
def print_face_with_landmarks(results,sample_image):

    face_landmarks = results.multi_face_landmarks

    # Importa le librerie di disegno da Mediapipe
    mp_drawing = mp.solutions.drawing_utils

    rgb_frame = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)

    if face_landmarks:
        for face_landmark in face_landmarks:
            # Disegna i landmarks sull'immagine
            mp_drawing.draw_landmarks(
                image=rgb_frame,
                landmark_list=face_landmark,
                connections=None, # senza connessioni
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,0,0), thickness=3, circle_radius=1),
                connection_drawing_spec=None
            )

    # Visualizza l'immagine con i landmarks sovrapposti
    plt.imshow(rgb_frame)
    plt.show()

image_name = "obama.jpg"
results, sample_image = detect_face(image_name)
print_face_with_landmarks(results, sample_image)

Di questi 478 landmarks che MediaPipe individua, quando la face-detection va a buon fine, siamo interessati soltanto a quelli associati all'area labiale. 

Il file LipLandmarks.py contiene un frozen-set di 20 coppie di landmarks di nostro interesse, landmarks coinvolti nell'area labiale.

L'insieme di coppie di landmarks definite nel file, considerando ogni coppia come un soggetto, delineano il contorno delle labbra.

Usiamo il file LipLandmark.py per disegnare i 20 segmenti associati alle 20 coppie di landmarks

In [ ]:
image_name = "assets/obama.jpg"

results, sample_image = detect_face(image_name)

height, width, _ = sample_image.shape

face_landmarks = results.multi_face_landmarks

# Importa le librerie di disegno da Mediapipe
mp_drawing = mp.solutions.drawing_utils

rgb_frame = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)

sample_image_landmarks = []
sample_list_of_euclidean_distances = []

iterator = iter(lp.lip_landmarks)
for j in range(0, len(lp.lip_landmarks)):
     i = next(iterator)
     if results and results.multi_face_landmarks:
        point = results.multi_face_landmarks[0].landmark[i[0]]
        node1_x = int(point.x * width)
        node1_y = int(point.y * height)
        sample_image_landmarks.append((node1_x,node1_y))
        cv2.circle(sample_image,(node1_x,node1_y),3,(0,0,255),1)

        point = results.multi_face_landmarks[0].landmark[i[1]]
        node2_x = int(point.x * width)
        node2_y = int(point.y * height)
        sample_image_landmarks.append((node2_x,node2_y))
        cv2.circle(sample_image,(node2_x,node2_y),3,(0,0,255),1)

        cv2.line(sample_image,(node1_x,node1_y),(node2_x,node2_y), (0,255,0), thickness=1)
        cv2.putText(sample_image, str(j+1), ((node1_x+node2_x)//2, (node1_y+node2_y)//2),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Calcolo della distanza euclidea tra i punti
        d = math.sqrt((node2_x - node1_x) ** 2 + (node2_y - node1_y) ** 2)
        sample_list_of_euclidean_distances.append(d)
        

plt.imshow(cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB))
plt.show()

for i, elem in enumerate(sample_list_of_euclidean_distances):
    print("Line ",i+1," lenght: ",elem)


Ogni coppia di landmark definisce quindi un segmento, come ad esempio quello del labbro inferiore o di quello superiore.

Secondo le nostre assunzioni, il contenuto biometricamente informativo risiede nella lunghezza di tali segmenti.

La lunghezza di un segmento è calcolata come la distanza euclidea tra i due landmarks in esso coinvolti.

---------------

# Codice per la creazione del dataset

Il nostro dataset è composto da video di soggetti in primo piano. Il nostro obiettivo è estrarne le caratteristiche biometriche dell'area labiale.

L'estrapolazione di tali caratteristiche è un'operazione che si applica ai singoli frame dei video.

Qui decidiamo, per ogni video, quanti frame considerare. Da ogni frame, verranno prelevate le caratteristiche biometriche delle labbra.

In [ ]:
num_frames = 20

La seguente funzione, fornendo come parametro il nome del file (video) sul quale operare, considera di tale video soltanto gli ultimi `num_frames` frame del video

Per ognuno di essi tenta la face detection quindi la lip feature extraction, cioè il calcolo della lunghezza dei 20 segmenti labbiali di notevole importanza biometrica.

Restituisce un vettore di vettore di 20 colonne (tanti quanto sono, appunto, le lip features in un frame cioè i segmenti) e tante righe quanti sono i frame dai quali è riuscito a prelevare le lip-features, il cui esito è strettamente correlato all'esito della face-detection

In [2]:
def ottieni_lista_distanze_euclidee(filename):
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh()

    cap = cv2.VideoCapture(filename)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - num_frames - 1)

    ret, frame = cap.read()

    landmarks = []

    list_of_euclidean_distances = []

    while ret:

        ret, frame = cap.read()
        if not ret:
            break

        height, width, _ = frame.shape
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = face_mesh.process(rgb_frame)

        iterator = iter(lp.lip_landmarks)
        for j in range(0, len(lp.lip_landmarks)):
            i = next(iterator)
            
            if results and results.multi_face_landmarks:
                # Primo elemento della tupla
                point = results.multi_face_landmarks[0].landmark[i[0]]
                node1_x = int(point.x * width)
                node1_y = int(point.y * height)
                landmarks.append((node1_x, node1_y))
                cv2.circle(frame, (node1_x, node1_y), 2, (255, 255, 255), -1)

                # Secondo elemento della tupla
                point = results.multi_face_landmarks[0].landmark[i[1]]
                node2_x = int(point.x * width)
                node2_y = int(point.y * height)
                landmarks.append((node2_x, node2_y))
                cv2.circle(frame, (node2_x, node2_y), 2, (255, 255, 255), -1)

                # Calcolo della distanza euclidea tra i punti
                d = math.sqrt((node2_x - node1_x) ** 2 + (node2_y - node1_y) ** 2)
                list_of_euclidean_distances.append(d)

    return list_of_euclidean_distances

Questa funzione [...]

In [3]:
def create_csv(csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)

        header_string = []
        for i in range(20):
            header_string.append("Feature " + str(i))
        header_string.append("Label")
        writer.writerow(header_string)

        os.chdir("Dataset")
        video_dir = os.listdir()

        for directory in video_dir:
            if os.path.isdir(directory):
                files = glob.glob(directory + "/*.avi")

                for video in tqdm(files, desc=directory, ncols=100):
                    res = ottieni_lista_distanze_euclidee(video)
                    # Se necessario, cambiare qui il metodo di prelievo della label
                    video_label = str(''.join(video.split("\\")[1].split(".")[0].split("_")[:4]))
                    if np.shape(res)[0] == 20 * num_frames:
                        res_split = np.array_split(res, num_frames)
                        for i in range(num_frames):
                            info_row = res_split[i]
                            writer.writerow(np.append(info_row, video_label))
        os.chdir("..")

Questa funzione [...]

In [4]:
def create_df(csv_filename):

    df = pandas.read_csv(csv_filename)
    df.sort_values(by=['Label'])

    x = df.iloc[:, :-1].values
    y = df['Label'].values

    j = 0
    last_label = y[0]
    y[0] = j

    for i in range(1, len(y)):
        to_print = str(y[i])
        if y[i] == last_label:
            y[i] = j
        else:
            last_label = y[i]
            j = j + 1
            y[i] = j

    x = scale(x)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

    data = {'x_train': x_train,
            'x_test': x_test,
            'y_train': y_train,
            'y_test': y_test}

    return data

Creiamo il dataset chiamando la funzione `create_csv(nome_csv_da_creare)`

In [ ]:
create_csv("dataset.csv")